# <center> Final model testing
# <center> News Lenta.ru dataset  
### <center> by Glazunov Artem

Works via Google Colab

Let's connect directories and load useful libs.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
cd /content/gdrive/'My Drive'/Colab_Notebooks/project_intern

/content/gdrive/My Drive/Colab_Notebooks/project_intern


In [3]:
import os
import pandas as pd
import gensim
from pprint import pprint

from sklearn.feature_extraction.text import CountVectorizer

from tqdm import tqdm_notebook

In [6]:
PATH_TO_RAW_DATA = 'data'
PATH_TO_DATA = 'texts_after_preproc'
PATH_TO_MODEL = 'results2'

In [5]:
data_preprocessed = pd.read_csv(os.path.join(PATH_TO_DATA,'texts_prep.csv'),index_col=0)
data_preprocessed 

,texts
0,бой сопоцкин друскеник заканчиваться отступлен...
1,министерство народный просвещение вид происход...
2,штабскапитан п н нестеров день увидеть район ж...
3,фотографкорреспондент daily mirror рассказыват...
4,лицо приезжать варшава люблин передавать туда ...
...,...
800965,певец сергей шнуров раскритиковывать свой колл...
800966,министерство юстиция россия предлагать изменят...
800967,испытание сша ранее запрещать договор ликвидац...
800968,близкий день европейский часть россия погода а...


Loading the model

In [7]:
LDA_model = gensim.models.LdaMulticore.load(os.path.join(PATH_TO_MODEL,'model2'))

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let's see the topics again.

In [8]:
pprint(LDA_model.print_topics())

[(0,
  '0.013*"москва" + 0.013*"город" + 0.011*"область" + 0.011*"тысяча" + '
  '0.010*"дом" + 0.009*"квартира" + 0.009*"метр" + 0.008*"район" + '
  '0.007*"площадь" + 0.007*"строительство"'),
 (1,
  '0.022*"фильм" + 0.012*"актер" + 0.010*"роль" + 0.009*"картина" + '
  '0.009*"режиссер" + 0.008*"премия" + 0.008*"актриса" + 0.007*"сериал" + '
  '0.007*"выходить" + 0.007*"главный"'),
 (2,
  '0.019*"президент" + 0.014*"глава" + 0.012*"владимир" + 0.011*"путин" + '
  '0.009*"сказать" + 0.008*"российский" + 0.007*"владимир путин" + '
  '0.006*"президент россия" + 0.006*"москва" + 0.006*"вопрос"'),
 (3,
  '0.015*"пользователь" + 0.013*"сеть" + 0.011*"компания" + 0.009*"сайт" + '
  '0.007*"новый" + 0.007*"опубликовывать" + 0.006*"видео" + 0.006*"появляться" '
  '+ 0.006*"информация" + 0.006*"сервис"'),
 (4,
  '0.011*"полиция" + 0.011*"происходить" + 0.010*"сотрудник" + '
  '0.008*"автомобиль" + 0.008*"задерживать" + 0.007*"мужчина" + '
  '0.007*"полицейский" + 0.006*"данные" + 0.006*"орган" +

Preprocessing...

In [9]:
data_list = data_preprocessed.texts.values.tolist()

In [11]:
vectorizer = CountVectorizer(ngram_range=(1, 2),min_df=640,max_df =0.25)
matrix = vectorizer.fit_transform(data_list)

In [12]:
def vect2gensim(vectorizer, dtmatrix):
     # transform sparse matrix into gensim corpus and dictionary
    corpus_vect_gensim = gensim.matutils.Sparse2Corpus(dtmatrix, documents_columns=False)
    dictionary = gensim.corpora.dictionary.Dictionary.from_corpus(corpus_vect_gensim,
        id2word=dict((id, word) for word, id in vectorizer.vocabulary_.items()))

    return (corpus_vect_gensim, dictionary)

In [13]:
def corpus2words_lists(gensim_corpus,gensim_dict):

  #Converts corpus to texts in the form of words lists for gensim Coherence model 

  dictionary = dict(gensim_dict)
  corpus_list = list(gensim_corpus)

  texts_new = []
  for text_meta in corpus_list:
    text_new=[]
    for pair in text_meta:
      for counter in range(1,pair[1]+1):
        text_new.append(dictionary[pair[0]])
    texts_new.append(text_new)

  return texts_new

In [14]:
(gensim_corpus, gensim_dict) = vect2gensim(vectorizer, matrix)

Let's give the titles to the topics. 

In [15]:
topics = {0:'Москва и городское хозяйство', 1:'Искусство, кинематограф и награды', 2: "Президент России и власть",
 3:"IT и медиа", 4:"Криминальная сводка",5:"Экономика и рынок", 6:"Вооружение и оборона", 7: "Шоубизнес, общее",
 8:"Наука и исследования",9:"Бизнес", 10:"Государство и законотворчество",11:"Перелеты и пассажирская авиация", 
 12: "Следствие, уголовное судопроизводство",  13:"Спорт", 14:"США",15:"Происшествия, общее",
 16:"Внешняя политика России, Украина",17:"Государство, парламент, выборы",18:"Банки и финансы",
 19:"Ближний восток, терроризм"}

In [16]:
topic_texts_descriptions = []
for text_vect in tqdm_notebook(gensim_corpus):
  topics_vect = LDA_model.get_document_topics(text_vect)

  topic_descr_text = []
  for pair in topics_vect:
    topic_descr_text.append((topics[pair[0]],pair[1]))

  topic_texts_descriptions.append(topic_descr_text)

topic_texts_descriptions[:3]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


[[('Москва и городское хозяйство', 0.09052615),
  ('Криминальная сводка', 0.02845486),
  ('Вооружение и оборона', 0.12142647),
  ('Шоубизнес, общее', 0.05804026),
  ('Наука и исследования', 0.11455826),
  ('Спорт', 0.07967457),
  ('Ближний восток, терроризм', 0.50607526)],
 [('Криминальная сводка', 0.07920003),
  ('Шоубизнес, общее', 0.4759003),
  ('Государство и законотворчество', 0.31652167),
  ('Спорт', 0.04213546),
  ('Ближний восток, терроризм', 0.082639255)],
 [('Москва и городское хозяйство', 0.024139423),
  ('Искусство, кинематограф и награды', 0.111133516),
  ('Криминальная сводка', 0.056608934),
  ('Наука и исследования', 0.17214194),
  ('Перелеты и пассажирская авиация', 0.14800431),
  ('Происшествия, общее', 0.30359754),
  ('Внешняя политика России, Украина', 0.023083046),
  ('Ближний восток, терроризм', 0.1605921)]]

In [17]:
data_raw = pd.read_csv(os.path.join(PATH_TO_RAW_DATA,'lenta-ru-news.csv'))
data_raw

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


Let's see topic descriptions of the texts.

In [18]:
data_res = data_raw[['text']].copy().dropna()
data_res.index = range(data_res.shape[0])

In [19]:
data_res['topics'] = topic_texts_descriptions 
data_res

,text,topics
0,Бои у Сопоцкина и Друскеник закончились отступ...,"[(Москва и городское хозяйство, 0.09052615), (..."
1,"Министерство народного просвещения, в виду про...","[(Криминальная сводка, 0.07920003), (Шоубизнес..."
2,"Штабс-капитан П. Н. Нестеров на днях, увидев в...","[(Москва и городское хозяйство, 0.024139423), ..."
3,Фотограф-корреспондент Daily Mirror рассказыва...,"[(Искусство, кинематограф и награды, 0.1211058..."
4,"Лица, приехавшие в Варшаву из Люблина, передаю...","[(Наука и исследования, 0.023595354), (Перелет..."
...,...,...
800965,Певец Сергей Шнуров раскритиковал свою коллегу...,"[(Искусство, кинематограф и награды, 0.1551859..."
800966,Министерство юстиции России предложило изменит...,"[(Президент России и власть, 0.012453331), (IT..."
800967,Испытание США ранее запрещенной Договором о ли...,"[(Вооружение и оборона, 0.41419792), (Перелеты..."
800968,В ближайшие дни в европейской части России пог...,"[(Москва и городское хозяйство, 0.62386787), (..."


In [32]:
data_res.values[5]

array(['Как стало известно агентству Ассошиэйтед Пресс, экипаж последней экспедиции на станцию "Мир" считает ее способной выйти из-под контроля.Командир Виктор Афанасьев сказал: "Мы чувствуем себя хорошо, но грустим, что оставили станцию летать в беспилотном режиме." Gazeta Ru подробно писала о том, что финансирование программы "Мир" закрыто, но конструкторы не спешат топить ее в Тихом океане, надеясь на изыскание внебюджетных средств.Они надеются, что правительство оплатит финальную экспедицию на "Мир" в феврале или марте 2000 года. Новая экспедиция должна постепенно перевести станцию на низкую орбиту, после чего 140- тонный "Мир" в основном сгорит в атмосфере, а незначительные его части упадут в Тихий океан. Но нет никакой гарантии, что пережившая столько отказов оборудования конструкция поведет себя адекватно командам. Афанасьев перечислил потенциально опасные элементы "Мира" на пресс- конференции, состоявшейся в понедельник в Звездном Городке. "Может упасть давление в кабине,- сказ

In [20]:
data_res.values[7]

array(['Япония приняла решение разморозить кредиты России на сумму в полтора миллиарда долларов. Об этом заявил сегодня после встречи с премьер-министром Японии вице-премьер правительства России Виктор Христенко. Он прибыл в Токио во главе делегации для участия в заседании  российско-японской межправительственной комиссии по торгово-экономическим  вопросам, которое  начнется в  среду  в Токио. Один из участников подготовки этого заседания сообщил "Интерфаксу", что Христенко постарается на конкретных макроэкономических показателях  за последние  месяцы доказать, что экономика России становится более стабильной и убедить Японию в своей платежеспособности и партнерской надежности. "Если на заседании  комиссии японская сторона будет удовлетворена этими данными, то многие российско-японские проекты по улучшению  экономики дальневосточного побережья, примут более реальные очертания", - подчеркнул источник. Сегодняшнее сообщение о разморозке кредита показывает, что приведенные Христенко выкла

In [21]:
data_res.values[300]

array(['МВД России называет имя человека, который использовал документы Лайпанова при подготовке взрывов в Москве. Об этом 17 сентября сообщила радиостанция "Эхо Москвы". Напомним, что человек, известный под именем Мухита Насировича Лайпанова, арендовал мебельный склад во взорванном доме на Каширском шоссе. И он же, по информации ГУВД, снимал помещение на первом этаже во взорваном доме в Печатниках. Однако еще 13 сентября министр внутренних дел РФ Рушайло заявил РИА "Новости", что фамилия Лайпанов - подставная. Настоящий Мухит Лайпанов погиб в феврале 1999 года в Ставропольском крае в результате ДТП. По сведениям МВД, под фамилией Лайпанова скрывался 30-летний уроженец Карачаевска  Ачемез Шагабанович Гочияев. Он объявлен в розыск. Особые приметы разыскиваемого: на пальцах рук может быть татуировка в виде перстня. Правоохранительные органы обратились с просьбой ко всем, кому известно его местонахождение, сообщить об этом по телефону в ближайшее отделение милиции.',
       list([('Москва

In [22]:
data_res.values[33000]

array(['В ночь со среды на четверг под Новосибирском при столкновении трех автомобилей - "Волги", "Газели" и  военного грузовика "Урал-375" - получил тяжелые травмы известный новосибирский режиссер, основатель и бессменный руководитель Городского драматического театра (НГДТ) Сергей Афанасьев. Как сообщили газете "Вечерний Новосибирск" в городской клинической больнице, у Сергея Афанасьева многочисленные переломы, но ожогов нет, он в сознании,  и шансов на благоприятный исход очень много. Однако ему предстоит провести на больничной койке не меньше 2 с половиной месяцев. Всего в этой катастрофе пострадали шесть человек, в том числе жена Афанасьева Татьяна. Ее состояние удовлетворительное. Режиссер и его супруга ехали в "Волге". Еще один пассажир легковой машины погиб, причем тело обгорело настолько, что не сразу удалось установить не только личность, но и то, что  это была женщина. Как пишет "Вечерний Новосибирск", погибшая занималась вопросами строительства нового здания драмтеатра. Серг

In [23]:
data_res.values[21000]

array(['Федеральная комиссия по рынку ценных бумаг России (ФКЦБ) подготовила поправки и дополнения в Уголовный и Уголовно-процессуальный кодексы, которые предусматривают уголовную ответственность за нарушения закона "О защите прав и законных интересов инвесторов на рынке ценных бумаг". Уголовная ответственность, в частности, устанавливается за утверждение отчета об итогах выпуска ценных бумаг, содержащего заведомо недостоверную информацию, за размещение эмиссионных ценных бумаг без госрегистрации их выпуска в случаях, когда такая регистрация является обязательной. Если эти деяния повлекли крупный ущерб, в 2000 раз превышающий минимальный размер оплаты труда (МРОТ), виновные наказываются штрафом 500 - 700 МРОТ или в размере доходов осужденного за 5 - 7 месяцев, либо обязательными работами на срок от 180 до 240 часов, либо исправительными работами на срок от 1 года до 2 лет. Те же деяния, совершенные неоднократно или совершенные группой лиц по сговору, наказываются лишением свободы на ср

In [24]:
data_res.values[2000]

array(['В минувший вторник таможенники столичного аэропорта "Шереметьево" задержали и вскрыли несколько ящиков, которые согласно накладным следовали из Румынии в Грузию. В ящиках были обнаружены гранатомет, четыре миномета, боеприпасы к ним, а также бронежилеты и камуфляжная форма. Поскольку транзитный провоз через Россию военного снаряжения запрещен, в отношении груза было заведено дело о нарушении таможенных правил. Как сообщает НТВ со ссылкой на "Интерфакс", следствию, в частности, предстоит выяснить, каким образом груз, который должен был следовать напрямую из Румынии в Грузию, оказался в России. Есть все основания полагать, что груз предназначался для Чечни, отметили в пресс-службе ГТК. Руководитель пресс-центра Минобороны Грузии Коба Ликликадзе опроверг сообщения ряда российских СМИ о том, что задержанный накануне на Шереметьевской таможне груз, направлявшийся транзитом в Тбилиси, якобы предназначался для чеченских боевиков. Министерство обороны Грузии будет добиваться возвращени

In [25]:
data_res.values[100000]

array(['Президент России назначил директора Федеральной службы по техническому и экспортному контролю Сергея Григорова председателем Межведомственной комиссии по защите государственной тайны. Одновременно президент своим указом освободил от исполнения обязанностей главы комиссии Бориса Алешина, который в правительстве Касьянова занимал пост вице-премьера России, а в настоящее время является руководителем Федерального агентства по промышленности. Текст указа был размещен 31 августа на официальном сайте президента. Правительству России поручено представить предложения по составу Межведомственной комиссии по защите гостайны по должностям, а также предложения по совершенствованию деятельности комиссии. Как сообщает информационное агентство ПРАЙМ-ТАСС, Григоров был назначен на пост директора Федеральной службы по техническому и экспортному контролю указом президента России в конце июля 2004 года. Ранее Григоров занимал должность председателя Государственной технической комиссии при президен

In [26]:
data_res.values[800966]

array(['Министерство юстиции России предложило изменить правила взыскания долгов. Соответствующий документ опубликован на портале проектов нормативных правовых актов. В частности, предлагается законодательно закрепить использование роботов-коллекторов — инструмента для отправки голосовых сообщений — для сбора задолженности с граждан. Уточняется, что такие роботы применяют систему генерации речи и способны поддерживать различные сценарии разговоров. Их использование будет считаться за взаимодействие с должником, однако он, в свою очередь, может отказаться от получения сообщений. Кроме того, документ предусматривает обязательный досудебный порядок возврата просроченных долгов физических лиц. Взыскатель может обратиться в суд только в том случае, если должник не расплатился в течение 30 дней после получения предложения о возврате долга. Также в документе говорится о том, что Правительством России могут быть установлены особенности взаимодействия с должником при возврате просроченной задол

In [27]:
data_res.values[800969]

array(['Ведущие футбольные чемпионаты ушли на зимние каникулы, но только не Английская премьер-лига (АПЛ). Самое время поучаствовать в совместном конкурсе букмекерской компании Olimp.Bet и тренажера игры на ставках social.bet «Новый год по-английски»! С 14 декабря 2019 года по 23 января 2020 года делайте виртуальные ставки на social.bet, добивайтесь максимальной прибыли и поднимайтесь в рейтинге участников. Призовой фонд конкурса — 100 тысяч рублей фрибетами, 11 подписок на ОККО Спорт, три футболки любимой команды АПЛ и мячи с автографами известных экспертов. В случае, если одинаковой максимальной прибыли в конкурсе достигнут два или более участников, выиграет тот, кто сделал больше ставок. Для участия в конкурсе необходимо зарегистрироваться на странице конкурса и в БК «Олимп». Отпразднуй «Новый год по-английски»!',
       list([('IT и медиа', 0.105140544), ('Экономика и рынок', 0.07844854), ('Шоубизнес, общее', 0.4033641), ('Наука и исследования', 0.07874211), ('Спорт', 0.18510184), 

In [28]:
data_res.values[500000]

array(['Компания Forbes, Inc. начала процесс отзыва лицензии на издание украинской версии журнала Forbes у компании UMH, принадлежащей предпринимателю Сергею Курченко, сообщает BuzzFeed. Также в Forbes заявили об увольнении из компании вице-президента Мигеля Форбса, который одобрил переход лицензии на издание журнала к Курченко в июне 2013 года. По словам представителя Forbes Мии Карбонелл (Mia Carbonell), увольнение Мигеля Форбса, о котором ранее в СМИ не сообщалось, произошло в конце 2013 года и не связано со скандалом вокруг украинской версии журнала. Причины, по которым было решено отозвать у UMH лицензию на выпуск Forbes на Украине, не называются. 6 марта 2014 года стало известно, что МВД Украины завело 11 уголовных дел в связи с деятельностью фирм, подконтрольных Сергею Курченко. Как заявил исполняющий обязанности главы украинского МВД Арсен Аваков, с 2012 по 2013 год компании Курченко ввезли на территорию Украины нефтепродукты с целью перепродажи за рубеж, однако в конечном итог

In [29]:
data_res.values[700000]

array(['Телеканал «Россия 24» в эфире программы «Вести» по ошибке назвал посла Украины в Таджикистане и Афганистане Виктора Никитюка одним из семи украинцев, погибших во время теракта в гостинице Intercontinental в Кабуле. Об этом сообщает таджикистанская радиостанция «Радио Озоди». По словам самого дипломата, в момент своей «гибели» он находился в Душанбе и крайне удивился новостям таджикских СМИ, которые ссылались на «Вести». Как утверждает Никитюк, он уверен, что сообщение о его гибели — провокация российского телеканала. «Ни одно из мировых средств массовой информации, кроме российского, не дало сообщения, что в ходе теракта погиб посол Украины», — пояснил он. О нападении боевиков на гостиницу Intercontinental сообщалось 21 января. По разным данным, погибли от 18 до 43 человек. Атака, длившаяся более 17 часов, была подавлена. Всю ночь, за исключением двухчасового перерыва, из гостиницы раздавались стрельба и взрывы. К утру на место происшествия прибыли отряды специального назначени

In [30]:
data_res.values[300000]

array(['Правительство РФ по предложению парламента Кемеровской области приняло решение о переводе региона в другой часовой пояс, сообщает "Интерфакс". 28 марта 2010 года, когда в стране будет осуществляться переход на летнее время, в Кемеровской области переводить часы не станут.\r Разница во времени между Кемерово и Москвой сократится с четырех (UTC+7, красноярское время) до трех часов (UTC+6, омское время). Кемеровская область окажется в одном часовом поясе с Новосибирском, Бишкеком, Барнаулом, Астаной и Алма-Атой.\r В 90-х годах многие российские регионы "подвинулись" на один часовой пояс в сторону Москвы. В частности, вся Западная Сибирь перешла с красноярского времени на омское. Кемеровская область оставалась единственным регионом, не осуществившим этот переход.',
       list([('Москва и городское хозяйство', 0.49382943), ('Шоубизнес, общее', 0.13095346), ('Государство и законотворчество', 0.3154704), ('Внешняя политика России, Украина', 0.058236096)])],
      dtype=object)

## Despite some degree of topic-text consistency, there are a lot of things to work on. In my opinion, further analysis could help  extract some seldom important topics. 